In [1]:
import numpy as np
from submit import my_map
from submit import my_fit
from submit import my_decode
import time as tm

ModuleNotFoundError: No module named 'submit'

In [2]:
Z_trn = np.loadtxt( "public_trn.txt" )
Z_tst = np.loadtxt( "public_tst.txt" )

n_trials = 5

d_size = 0
t_train = 0
t_map = 0
acc = 0

In [3]:
for t in range( n_trials ):
  tic = tm.perf_counter()
  w, b = my_fit( Z_trn[:, :-1], Z_trn[:,-1] )
  toc = tm.perf_counter()

  t_train += toc - tic
  w = w.reshape( -1 )

  d_size += w.shape[0]

  tic = tm.perf_counter()
  feat = my_map( Z_tst[:, :-1] )
  toc = tm.perf_counter()
  t_map += toc - tic

  scores = feat.dot( w ) + b

  pred = np.zeros_like( scores )
  pred[ scores > 0 ] = 1
  print(t," = ", np.average( Z_tst[ :, -1 ] == pred ))
  acc += np.average( Z_tst[ :, -1 ] == pred )

0  =  1.0
1  =  1.0
2  =  1.0
3  =  1.0
4  =  1.0


In [4]:
d_size /= n_trials
t_train /= n_trials
t_map /= n_trials
acc /= n_trials

In [5]:
print(f"Average Accuracy over {n_trials} trials: {acc * 100:.2f}%")
print(f"Average Model Size: {d_size}")
print(f"Average Training Time: {t_train:.4f} seconds")
print(f"Average Mapping Time: {t_map:.4f} seconds")

Average Accuracy over 5 trials: 100.00%
Average Model Size: 255.0
Average Training Time: 0.1373 seconds
Average Mapping Time: 0.0107 seconds


In [10]:
def get_model( p, q, r, s ):
  p = np.maximum( p, 0 )
  q = np.maximum( q, 0 )
  r = np.maximum( r, 0 )
  s = np.maximum( s, 0 )
  d = p - q
  c = r - s
  alpha = ( d + c ) / 2
  beta = ( d - c ) / 2
  w = np.zeros( ( len( alpha ) + 1, )  )
  w[:-1] += alpha
  w[1:] += beta
  return w

In [11]:
W = np.loadtxt( "public_mod.txt" )
( n_models, dims ) = W.shape
t_decode = 0
m_dist = 0
for t in range( n_trials ):
  for itr in range( n_models ):
    w = W[ itr, : ]
    tic = tm.perf_counter()
    p_hat, q_hat, r_hat, s_hat = my_decode( w )
    toc = tm.perf_counter()
    t_decode += toc - tic
    w_hat = get_model( p_hat, q_hat, r_hat, s_hat )
    m_dist += np.linalg.norm( w - w_hat )

In [12]:
t_decode /= ( n_trials * n_models )
m_dist /= ( n_trials * n_models )

In [13]:
print( f"{d_size},{t_train},{t_map},{1 - acc},{t_decode},{m_dist}" )

255.0,0.1373432410000021,0.010705083600009857,0.0,0.0005727078000012397,4.536370440084288e-16


In [18]:
from sympy import symbols, simplify

# Define symbols
c = symbols('c0:8')
p = symbols('p0:8')
q = symbols('q0:8')
r = symbols('r0:8')
s = symbols('s0:8')
# p_ = symbols('p_0:8')
# q_ = symbols('q_0:8')
# r_ = symbols('r_0:8')
# s_ = symbols('s_0:8')

# Initialize Delta arrays
Delta = [0] * 8
Delta_ = [0] * 8

# Base case
Delta[0]  = (1 - c[0]) * (p[0]) + c[0] * (s[0])
Delta_[0] = (1 - c[0]) * (q[0]) + c[0] * (r[0])

# Recurrence
for i in range(1, 8):
    print("Starting,", i)
    Delta[i]  = simplify((1 - c[i]) * (Delta[i-1] + p[i]) + c[i] * (Delta_[i-1] + s[i]))
    Delta_[i] = simplify((1 - c[i]) * (Delta_[i-1] + q[i]) + c[i] * (Delta[i-1] + r[i]))

# Final result
Delta_7 = Delta[7]
Delta_7_ = Delta_[7]

# Now you can print or export Delta_7 and Delta_7_ for further analysis.
Delta_7

Starting, 1
Starting, 2
Starting, 3
Starting, 4
Starting, 5
Starting, 6
Starting, 7


c7*(c6*(c5*(c4*(c3*(c2*(c1*(c0*r0 - q0*(c0 - 1) + s1) + r2 - (c1 - 1)*(c0*s0 - p0*(c0 - 1) + p1)) + s3 - (c2 - 1)*(c1*(c0*s0 - p0*(c0 - 1) + r1) + q2 - (c1 - 1)*(c0*r0 - q0*(c0 - 1) + q1))) + r4 - (c3 - 1)*(c2*(c1*(c0*s0 - p0*(c0 - 1) + r1) + s2 - (c1 - 1)*(c0*r0 - q0*(c0 - 1) + q1)) + p3 - (c2 - 1)*(c1*(c0*r0 - q0*(c0 - 1) + s1) + p2 - (c1 - 1)*(c0*s0 - p0*(c0 - 1) + p1)))) + s5 - (c4 - 1)*(c3*(c2*(c1*(c0*s0 - p0*(c0 - 1) + r1) + s2 - (c1 - 1)*(c0*r0 - q0*(c0 - 1) + q1)) + r3 - (c2 - 1)*(c1*(c0*r0 - q0*(c0 - 1) + s1) + p2 - (c1 - 1)*(c0*s0 - p0*(c0 - 1) + p1))) + q4 - (c3 - 1)*(c2*(c1*(c0*r0 - q0*(c0 - 1) + s1) + r2 - (c1 - 1)*(c0*s0 - p0*(c0 - 1) + p1)) + q3 - (c2 - 1)*(c1*(c0*s0 - p0*(c0 - 1) + r1) + q2 - (c1 - 1)*(c0*r0 - q0*(c0 - 1) + q1))))) + r6 - (c5 - 1)*(c4*(c3*(c2*(c1*(c0*s0 - p0*(c0 - 1) + r1) + s2 - (c1 - 1)*(c0*r0 - q0*(c0 - 1) + q1)) + r3 - (c2 - 1)*(c1*(c0*r0 - q0*(c0 - 1) + s1) + p2 - (c1 - 1)*(c0*s0 - p0*(c0 - 1) + p1))) + s4 - (c3 - 1)*(c2*(c1*(c0*r0 - q0*(c0 - 1) + 

In [23]:
from sympy import expand

# Fully expand Delta_7
expanded = expand(Delta[2])

# Count number of additive terms
num_terms = len(expanded.as_ordered_terms())
print(f"Number of additive terms in Delta_2: {num_terms}")

Number of additive terms in Delta_2: 33


In [22]:
Delta[2]

c2*(c1*(c0*s0 - p0*(c0 - 1) + r1) + s2 - (c1 - 1)*(c0*r0 - q0*(c0 - 1) + q1)) - (c2 - 1)*(c1*(c0*r0 - q0*(c0 - 1) + s1) + p2 - (c1 - 1)*(c0*s0 - p0*(c0 - 1) + p1))

In [31]:
from sympy import symbols, simplify, collect, expand

# Define symbols
c = symbols('c0:8')
p = symbols('p0:8')
q = symbols('q0:8')
r = symbols('r0:8')
s = symbols('s0:8')

# Initialize Delta arrays
Delta = [0] * 8
Delta_ = [0] * 8

# Base case
Delta[0]  = (1 - c[0]) * (p[0]) + c[0] * (s[0])
Delta_[0] = (1 - c[0]) * (q[0]) + c[0] * (r[0])

# Recurrence
for i in range(1, 3):
    Delta[i]  = simplify((1 - c[i]) * (Delta[i-1] + p[i]) + c[i] * (Delta_[i-1] + s[i]))
    Delta_[i] = simplify((1 - c[i]) * (Delta_[i-1] + q[i]) + c[i] * (Delta[i-1] + r[i]))

# Expand and collect like terms for Delta[7]
Delta_7 = expand(Delta[2])
terms_combined = collect(Delta_7, p + q + r + s, evaluate=False)

terms_combined

{p0: -2*c0*c1*c2 + c0*c1 + c0*c2 - c0 + 2*c1*c2 - c1 - c2 + 1,
 p1: c1*c2 - c1 - c2 + 1,
 p2: 1 - c2,
 s0: 2*c0*c1*c2 - c0*c1 - c0*c2 + c0,
 q0: 2*c0*c1*c2 - c0*c1 - c0*c2 - 2*c1*c2 + c1 + c2,
 s1: -c1*c2 + c1,
 q1: -c1*c2 + c2,
 s2: c2,
 r0: -2*c0*c1*c2 + c0*c1 + c0*c2,
 r1: c1*c2}

In [33]:
from sympy import symbols, simplify, collect, expand

# Define symbols
k = 7  # You can change this to any value up to 7
c = symbols(f'c0:{k+1}')
c_comps = symbols(f'c_comps0:{k+1}')
p = symbols(f'p0:{k+1}')
q = symbols(f'q0:{k+1}')
r = symbols(f'r0:{k+1}')
s = symbols(f's0:{k+1}')

# Initialize Delta arrays
Delta = [0] * (k+1)
Delta_ = [0] * (k+1)

# Base case
Delta[0]  = c_comps[0] * p[0] + c[0] * s[0]
Delta_[0] = c_comps[0] * q[0] + c[0] * r[0]

# Recurrence
for i in range(1, k+1):
    Delta[i]  = simplify(c_comps[i] * (Delta[i-1] + p[i]) + c[i] * (Delta_[i-1] + s[i]))
    Delta_[i] = simplify(c_comps[i] * (Delta_[i-1] + q[i]) + c[i] * (Delta[i-1] + r[i]))

# Expand and collect like terms for Delta[k]
Delta_k = expand(Delta[k])
grouped_terms = collect(Delta_k, p + q + r + s, evaluate=False)

# Print coefficients for each term
print(f"Coefficients for Delta[{k}]:\n")
for symbol in p + q + r + s:
    if symbol in grouped_terms:
        print(f"{symbol}: {grouped_terms[symbol]}")

Coefficients for Delta[7]:

p0: c1*c2*c3*c4*c5*c6*c_comps0*c_comps7 + c1*c2*c3*c4*c5*c7*c_comps0*c_comps6 + c1*c2*c3*c4*c6*c7*c_comps0*c_comps5 + c1*c2*c3*c4*c_comps0*c_comps5*c_comps6*c_comps7 + c1*c2*c3*c5*c6*c7*c_comps0*c_comps4 + c1*c2*c3*c5*c_comps0*c_comps4*c_comps6*c_comps7 + c1*c2*c3*c6*c_comps0*c_comps4*c_comps5*c_comps7 + c1*c2*c3*c7*c_comps0*c_comps4*c_comps5*c_comps6 + c1*c2*c4*c5*c6*c7*c_comps0*c_comps3 + c1*c2*c4*c5*c_comps0*c_comps3*c_comps6*c_comps7 + c1*c2*c4*c6*c_comps0*c_comps3*c_comps5*c_comps7 + c1*c2*c4*c7*c_comps0*c_comps3*c_comps5*c_comps6 + c1*c2*c5*c6*c_comps0*c_comps3*c_comps4*c_comps7 + c1*c2*c5*c7*c_comps0*c_comps3*c_comps4*c_comps6 + c1*c2*c6*c7*c_comps0*c_comps3*c_comps4*c_comps5 + c1*c2*c_comps0*c_comps3*c_comps4*c_comps5*c_comps6*c_comps7 + c1*c3*c4*c5*c6*c7*c_comps0*c_comps2 + c1*c3*c4*c5*c_comps0*c_comps2*c_comps6*c_comps7 + c1*c3*c4*c6*c_comps0*c_comps2*c_comps5*c_comps7 + c1*c3*c4*c7*c_comps0*c_comps2*c_comps5*c_comps6 + c1*c3*c5*c6*c_comps0*c_comps

In [34]:
from sympy import symbols, simplify, collect, expand, Mul

# Set k
k = 2  # You can change this up to 7

# Define symbols
c = symbols(f'c0:{k+1}')
c_comps = symbols(f'c_comps0:{k+1}')
p = symbols(f'p0:{k+1}')
q = symbols(f'q0:{k+1}')
r = symbols(f'r0:{k+1}')
s = symbols(f's0:{k+1}')

# Initialize Delta arrays
Delta = [0] * (k+1)
Delta_ = [0] * (k+1)

# Base case
Delta[0]  = c_comps[0] * p[0] + c[0] * s[0]
Delta_[0] = c_comps[0] * q[0] + c[0] * r[0]

# Recurrence
for i in range(1, k+1):
    Delta[i]  = simplify(c_comps[i] * (Delta[i-1] + p[i]) + c[i] * (Delta_[i-1] + s[i]))
    Delta_[i] = simplify(c_comps[i] * (Delta_[i-1] + q[i]) + c[i] * (Delta[i-1] + r[i]))

# Expand and collect like terms for Delta[k]
Delta_k = expand(Delta[k])
grouped_terms = collect(Delta_k, p + q + r + s, evaluate=False)

# Extract all coefficients and normalize their factors
unique_products = set()

for expr in grouped_terms.values():
    if isinstance(expr, Mul):
        factors = expr.as_ordered_factors()
    else:
        factors = [expr]
    flat_factors = []
    for factor in factors:
        flat_factors.extend(factor.as_ordered_factors() if isinstance(factor, Mul) else [factor])
    normalized = tuple(sorted(flat_factors, key=str))
    unique_products.add(normalized)

# Convert set of tuples back to expressions
unique_exprs = [Mul(*factors) for factors in unique_products]

len(unique_exprs), unique_exprs

(10,
 [c_comps2,
  c2*c_comps1,
  c2,
  c0*c1*c_comps2 + c0*c2*c_comps1,
  c1*c_comps0*c_comps2 + c2*c_comps0*c_comps1,
  c1*c2*c_comps0 + c_comps0*c_comps1*c_comps2,
  c0*c1*c2 + c0*c_comps1*c_comps2,
  c1*c_comps2,
  c1*c2,
  c_comps1*c_comps2])